# Make Model Data Set
Add noise to model visibilities and make a data set with the same shape as the real data.

In [1]:
import numpy as np
import h5py

import pandas as pd
from scipy.interpolate import interp1d, interp2d

## Load Data

In [ ]:
for trclass in ["EQ14", "EQ28"]:
    for field in ["A", "B", "C", "E"]: 
        print(trclass, field)

        std_path = "/users/pkeller/code/H1C_IDR3.2/data/noise_std.h5"
        vis_gleam_path = f"/users/pkeller/code/ClosureSim/data/vis_{trclass}_F{field}B2.h5"
        vis_bright_path = f"/users/pkeller/code/ClosureSim/data/vis_bright_{trclass}_F{field}B2.h5"
        data_path = f"/lustre/aoc/projects/hera/pkeller/data/H1C_IDR3.2/sample/{trclass}_F{field}_B2.h5"

        with h5py.File(vis_gleam_path) as f:
            lstm = f["LST"][()]
            freqm = f["Frequencies"][()]
            vis = f["Visibilities"][()]

        with h5py.File(vis_bright_path) as f:
            vis += f["Visibilities"][()]

        with h5py.File(data_path) as f:
            data = f["bispec"][()]
            freqd = f["FRQ"][()]
            lstd = f["LST"][()]

        with h5py.File(std_path) as f:
            lstn = f["LST"][()]
            stdn = f["RMS"][()].real

        # interpolate model visibilities along frequency and time axis
        vis = interp1d(freqm, vis, axis=-1, fill_value="extrapolate")(freqd)
        vis = interp1d(lstm, vis, axis=-2, fill_value="extrapolate")(lstd)
        
        # interpolate NaN-values
        stdn = np.sqrt(np.nanmean(stdn**2, axis=(0,)))
        stdn = pd.DataFrame(stdn).interpolate(axis=1).values
        stdn = pd.DataFrame(stdn).interpolate(axis=0).values
        stdn = stdn[np.where(~np.isnan(stdn).all(-1))]
        lstn = lstn[np.where(~np.isnan(stdn).all(-1))]
        stdf = interp2d(freqd, lstn, stdn)

        std = np.zeros_like(vis).real

        for i, lst in enumerate(lstd):
            for j, freq in enumerate(freqd):
                std[:, :, i, j] = stdf(freq, lst)
                
        bispec = np.zeros_like(data)

        for i in range(data.shape[1]):
            for j in range(data.shape[2]):
                visn = vis + np.random.normal(np.zeros_like(vis).real, std) + 1j * np.random.normal(np.zeros_like(vis).real, std)
                bispec[:, i, j] = np.product(visn, axis=0)
        
        bispec[np.where(np.isnan(data))] = np.nan

        with h5py.File(data_path, "a") as f:
            dname = f"bispec model"
            if dname in f.keys():
                del f[dname]
            f.create_dataset(f"bispec model", data=bispec)

EQ14 A


/lustre/aoc/projects/hera/pkeller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: Mean of empty slice


EQ14 B


## Make Data Set

In [182]:
data = np.zeros_like(data)

for i in range(cph.shape[1]):
    for j in range(cph.shape[2]):
        visn = vis + np.random.normal(np.zeros_like(vis).real, std) + 1j * np.random.normal(np.zeros_like(vis).real, std)
        data[:, i, j] = np.product(visn, axis=0)

## Save to File

In [184]:
with h5py.File(data_path, "w") as f:
    f.create_dataset(f"bispec model", data=data)